In [1]:
from music21 import converter, instrument, note, chord
import glob
import pickle
import keras 
import numpy
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint



c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
 def train_network():
#     notes = get_notes()
#     n_vocab = len(set(notes))
    
#     network_input, network_output = prepare_sequence

SyntaxError: unexpected EOF while parsing (<ipython-input-2-f798d2caf55d>, line 5)

In [3]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100   #det här kan vi gärna ändra
    pitchnames = sorted(set(item for item in notes))
    
    note_to_int = dict((note,number) for number,note in enumerate(pitchnames))
    
    network_input = []
    network_output = []
    
    for i in range(0,len(notes)-sequence_length,1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i+sequence_length]
        
        network_input.append([note_to_int[char] for char in sequence_in ])
        
        network_output.append(note_to_int[sequence_out])
        
    n_patterns = len(network_input)
    
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length,1))
    
    network_input = network_input /float(n_vocab)
    
    network_output = np_utils.to_categorical(network_output)
    
    
    return(network_input, network_output)

In [4]:
def get_notes():
    notes=[]
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        
        parts = instrument.partitionByInstrument(midi)



        if parts:
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
            
            
        for element in notes_to_parse:
            #den ena är accord, t.ex. B5, den andra är siffror, t.ex 11.2
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))

            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                #print('.'.join(str(n) for n in element.normalOrder))
                

        with open('data/notes','wb') as filepath:
            pickle.dump(notes, filepath)
            
    return notes
                
                


In [5]:
def create_network(network_input, n_vocab):
    
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1],network_input.shape[2]),return_sequences=True))    
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
    
    return model
    

In [6]:
def train(model,network_input,network_output):
    filepath = "weights-improvement-{epoch:02d}--{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor = 'loss',
        verbose = 0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]
    
    model.fit(network_input, network_output, epochs=100,batch_size = 64, callbacks=callbacks_list)
    
    
    return

In [8]:
notes = get_notes()
n_vocab = len(set(notes))
network_input,network_output = prepare_sequences(notes,n_vocab)

model = create_network(network_input,n_vocab)
train(model,network_input,network_output)

Epoch 1/100
5370/5370 [==============================] - 168s 31ms/step - loss: 4.5632
Epoch 2/100
5370/5370 [==============================] - 179s 33ms/step - loss: 4.3700
Epoch 3/100
5370/5370 [==============================] - 174s 32ms/step - loss: 4.2771
Epoch 4/100
5370/5370 [==============================] - 178s 33ms/step - loss: 4.2333
Epoch 5/100
5370/5370 [==============================] - 175s 33ms/step - loss: 4.1722
Epoch 6/100
5370/5370 [==============================] - 180s 34ms/step - loss: 4.1161
Epoch 7/100
5370/5370 [==============================] - 171s 32ms/step - loss: 4.0283
Epoch 8/100
5370/5370 [==============================] - 169s 31ms/step - loss: 3.9168
Epoch 9/100
5370/5370 [==============================] - 166s 31ms/step - loss: 3.7680
Epoch 10/100
5370/5370 [==============================] - 167s 31ms/step - loss: 3.6260
Epoch 11/100
5370/5370 [==============================] - 171s 32ms/step - loss: 3.4444
Epoch 12/100
5370/5370 [=================

5370/5370 [==============================] - 164s 31ms/step - loss: 0.0550
Epoch 95/100
5370/5370 [==============================] - 164s 30ms/step - loss: 0.0493
Epoch 96/100
5370/5370 [==============================] - 164s 31ms/step - loss: 0.0402
Epoch 97/100
5370/5370 [==============================] - 165s 31ms/step - loss: 0.0312
Epoch 98/100
5370/5370 [==============================] - 165s 31ms/step - loss: 0.0692
Epoch 99/100
5370/5370 [==============================] - 164s 31ms/step - loss: 0.0446
Epoch 100/100
5370/5370 [==============================] - 164s 31ms/step - loss: 0.0277
